# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [3]:
from backend import data_paths
from backend import evaluation_utils
from backend import gauge_groups_utils
from backend import loading_utils
from backend import metrics_utils
from backend import skill_prediction_utils

# Load Basin Attributes for Latitude & Longitude

In [4]:
gauges = gauge_groups_utils.get_full_gauge_group()
print(f'There are {len(gauges)} gauges.')

In [5]:
attributes = loading_utils.load_attributes_file(gauges=gauges)

# Load Return Period Metrics

In [6]:
_DATASET_RETURN_PERIOD_METRICS_PATH = {
    'google_2014': data_paths.GOOGLE_2014_RETURN_PERIOD_METRICS_DIR,
    'google_1980': data_paths.GOOGLE_1980_RETURN_PERIOD_METRICS_DIR,
    'glofas_2014': data_paths.GLOFAS_2014_RETURN_PERIOD_METRICS_DIR,
    'glofas_1980': data_paths.GLOFAS_1980_RETURN_PERIOD_METRICS_DIR,
}

In [7]:
precisions_by_lead_time = {}
recalls_by_lead_time = {}

precisions_by_return_period = {}
recalls_by_return_period = {}

for dataset, data_path in _DATASET_RETURN_PERIOD_METRICS_PATH.items():
    print(f'Working on {dataset} ...')
    file_path = data_paths.CONCATENATED_RETURN_PERIOD_DICTS_DIR / f'{dataset}_return_period_dicts.pkl'
    with open(file_path, 'rb') as f:
        precisions_by_lead_time[dataset], recalls_by_lead_time[dataset] = pkl.load(f)
    print(f'Finished loading {dataset}. \n')

# Calculate F1 Scores from Precision & Recall

In [8]:
f1s_by_lead_time = {
    dataset: {
        experiment: {
            lead_time:
              evaluation_utils.f1_from_precision_and_recall_dfs(
                  precision_df=precisions_by_lead_time[dataset][experiment][lead_time],
                  recall_df=recalls_by_lead_time[dataset][experiment][lead_time]
              ) for lead_time in data_paths.LEAD_TIMES
        } for experiment in precisions_by_lead_time[dataset]
    } for dataset in _DATASET_RETURN_PERIOD_METRICS_PATH
}

# Skill Predictability

## Extract Basin Attributes as Classifier/Regression Inputs

In [9]:
# A subset of attributes to use as predictor values. This selection is mostly
# due to challenges in naming all the HydroATLAS variables, and the skill
# is not affected by removing some of the landcover class variables.
ATTRIBUTE_DESCRIPTIVE_NAMES = {
    'calculated_drain_area': 'Drain Area',
    'inu_pc_umn': 'Inundation Percent Min',
    'inu_pc_umx': 'Inundation Percent Max',
    # 'inu_pc_ult': 'Inundation Percent Long Term Maximum',
    'lka_pc_use': 'Precent Lake Area',
    'lkv_mc_usu': 'Lake Volume',
    'rev_mc_usu': 'Reservoir Volume',
    'ria_ha_usu': 'River Area',
    'riv_tc_usu': 'River Volume',
    'ele_mt_uav': 'Elevation',
    'slp_dg_uav': 'Slope',
    'tmp_dc_uyr': 'Air Temperature',
    'pre_mm_uyr': 'Precipitation',
    'pet_mm_uyr': 'PET',
    'aet_mm_uyr': 'AET',
    'ari_ix_uav': 'Aridity Index',
    'cmi_ix_uyr': 'Climate Moisture Index',
    'snw_pc_uyr': 'Snow Cover Extent',
    'for_pc_use': 'Forest Cover Extent',
    'crp_pc_use': 'Cropland Extent',
    'pst_pc_use': 'Pastiure Extent',
    'ire_pc_use': 'Irrigated Area Extent',
    'gla_pc_use': 'Glacier Extent',
    'prm_pc_use': 'Permafrost Extent',
    'pac_pc_use': 'Protected Area Extent',
    'cly_pc_uav': 'Soil Clay Fraction',
    'slt_pc_uav': 'Soil Silt Fraction',
    'snd_pc_uav': 'Soil Sand Fraction',
    'soc_th_uav': 'Soil Organic Carbon',
    'swc_pc_uyr': 'Soil Water Content',
    'kar_pc_use': 'Karst Area Extent',
    'ero_kh_uav': 'Soil Erosion',
    'pop_ct_usu': 'Population Count',
    'ppd_pk_uav': 'Population Density',
    'urb_pc_use': 'Urban Area Extent',
    'nli_ix_uav': 'Nighttime Lights Index',
    # 'rdd_mk_uav': 'Road Density',
    # 'hft_ix_u93': 'Human Footprint 1993',
    # 'hft_ix_u09': 'Human Footprint 2009',
    'gdp_ud_usu': 'GDP',
    # 'latitude': 'latitude',
    # 'longitude': 'longitude',
}

# Select the subset of attributes.
regression_attributes = attributes[ATTRIBUTE_DESCRIPTIVE_NAMES.keys()]
regression_attributes.rename(columns=ATTRIBUTE_DESCRIPTIVE_NAMES, inplace=True)

# Normalize the attributes.
regression_attributes = (regression_attributes - regression_attributes.mean()) / regression_attributes.std()

## Hyperparameters for Reliability Score Predictions

In [10]:
# Hyperparamters for reliability score predictions.
lead_time = 0
return_period = 5
metric='F1 Score'

# Feature Importances

In [11]:
fig, axes = plt.subplots(
    2, 1, 
    figsize=(
        evaluation_utils.NATURE_FIG_SIZES['two_column'],
        evaluation_utils.NATURE_FIG_SIZES['two_column'],
    )
)

# --- GloFAS --------------------------------
regression_data = f1s_by_lead_time['glofas_1980'][metrics_utils.GLOFAS_VARIABLE][
    lead_time][return_period].dropna().rename(metric)
experiment = metrics_utils.GLOFAS_VARIABLE
bins = None #[-0.1, regression_data.mean(), 1.01]

x, y = skill_prediction_utils.make_predictors(
    x=regression_attributes,
    y=regression_data,
    metric=metric,
    bins=bins
)

glofas_importances, _ = y_hat = skill_prediction_utils.feature_importances(
    x, y,
    # classifier=True
    regression=True
)

skill_prediction_utils.plot_feature_importances(
    importances=glofas_importances,
    ax=axes[0]
)
axes[0].set_title(
    evaluation_utils.EXPERIMENT_NAMES[experiment],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['title']
)

# --- Google Model ----------------------------
regression_data = f1s_by_lead_time['google_1980']['kfold_splits'][
    lead_time][return_period].dropna().rename(metric)
experiment = 'kfold_splits'
bins = None #[-0.1, regression_data.mean(), 1.01]

x, y = skill_prediction_utils.make_predictors(
    x=regression_attributes,
    y=regression_data,
    metric=metric,
    bins=bins
)

google_importances, _ = y_hat = skill_prediction_utils.feature_importances(
    x, y,
    # classifier=True
    regression=True
)

skill_prediction_utils.plot_feature_importances(
    importances=google_importances,
    ax=axes[1]
)
axes[1].set_title(
    evaluation_utils.EXPERIMENT_NAMES[experiment], 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['title']
)

plt.tight_layout()

evaluation_utils.save_figure(data_paths.PREDICTABILITY_FEATURE_IMPORTANCES_FILENAME)

In [12]:
# Choose top 5 most important features from both models.
num_importances = 5
top_features = list(set(glofas_importances.sort_values(ascending=False).index[:num_importances]).union(
    set(google_importances.sort_values(ascending=False).index[:num_importances])))
top_features

## F1 Score Predictions

In [13]:
fig, axes = plt.subplots(
    2, 2, 
    figsize=(
            evaluation_utils.NATURE_FIG_SIZES['one_half_column'],
            evaluation_utils.NATURE_FIG_SIZES['one_half_column']
    )
)

gs = axes[0, 0].get_gridspec()
for ax in axes[:, -1]:
    ax.remove()
bigax = fig.add_subplot(gs[:, -1])

# --- GloFAS --------------------------------
regression_data = f1s_by_lead_time['glofas_1980'][metrics_utils.GLOFAS_VARIABLE][
    lead_time][return_period].dropna().rename(metric)
experiment = metrics_utils.GLOFAS_VARIABLE
bins = None# [-0.1, regression_data.mean(), 1.01]

x, y = skill_prediction_utils.make_predictors(
    x=regression_attributes,
    y=regression_data,
    metric=metric,
    bins=bins
)

y_hat = skill_prediction_utils.train_kfold(
    x, y,
    # classifier=True
    regression=True
)

bins = [-0.1, regression_data.mean(), 1.01]
bin_names = [idx for idx, bin in enumerate(bins[1:])]
y_cat = pd.cut(y, bins=bins, include_lowest=True, labels=bin_names)
y_hat_cat = pd.cut(y_hat, bins=bins, include_lowest=True, labels=bin_names)

print('------------------------------------')
print('------------------------------------')
print(experiment)
print(precision_recall_fscore_support(y_cat, y_hat_cat, average='micro'))
print('------------------------------------')
print('------------------------------------')

ax = axes[0, 0]
cm = confusion_matrix(y_cat, y_hat_cat, normalize='true')
sns.heatmap(cm, annot=True, ax=ax, cbar=False, fmt='.2g')
ax.set_aspect('equal', 'box')
ax.set_title(
    f'{evaluation_utils.EXPERIMENT_NAMES[experiment]} (N = {y.shape[0]})',
    fontsize=evaluation_utils.NATURE_FONT_SIZES['title']
)
ax.set_xlabel(
    'Predicted Labels', 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['axis_label']
)
ax.set_ylabel(
    'True Labels', 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['axis_label']
)
ax.xaxis.set_ticklabels(
    ['Below Mean', 'Above Mean'],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)
ax.yaxis.set_ticklabels(
    ['Below Mean', 'Above Mean'],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)


# --- Google Model ----------------------------
regression_data = f1s_by_lead_time['google_1980']['kfold_splits'][
    lead_time][return_period].dropna().rename(metric)
experiment = 'kfold_splits'
bins = None# [-0.1, regression_data.mean(), 1.01]

x, y = skill_prediction_utils.make_predictors(
    x=regression_attributes,
    y=regression_data,
    metric=metric,
    bins=bins
)

y_hat = skill_prediction_utils.train_kfold(
    x, y,
    # classifier=True
    regression=True
)

bins = [-0.1, regression_data.mean(), 1.01]
bin_names = [idx for idx, bin in enumerate(bins[1:])]
y_cat = pd.cut(y, bins=bins, include_lowest=True, labels=bin_names)
y_hat_cat = pd.cut(y_hat, bins=bins, include_lowest=True, labels=bin_names)

print('------------------------------------')
print('------------------------------------')
print(experiment)
print(precision_recall_fscore_support(y_cat, y_hat_cat, average='micro'))
print('------------------------------------')
print('------------------------------------')

ax = axes[1, 0]
cm = confusion_matrix(y_cat, y_hat_cat, normalize='true')
sns.heatmap(cm, annot=True, ax=ax, cbar=False, fmt='.2g')
ax.set_aspect('equal', 'box')
ax.set_title(
    f'{evaluation_utils.EXPERIMENT_NAMES[experiment]} (N = {y.shape[0]})',
    fontsize=evaluation_utils.NATURE_FONT_SIZES['title']
)
ax.set_xlabel(
    'Predicted Labels', 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['axis_label']
)
ax.set_ylabel(
    'True Labels', 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['axis_label']
)
ax.xaxis.set_ticklabels(
    ['Below Mean', 'Above Mean'],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)
ax.yaxis.set_ticklabels(
    ['Below Mean', 'Above Mean'],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)

# --- Attribute Correlations -----------------
combined_data = pd.concat([
    f1s_by_lead_time['glofas_1980'][metrics_utils.GLOFAS_VARIABLE][
        lead_time][return_period].dropna().rename(metrics_utils.GLOFAS_VARIABLE),
    f1s_by_lead_time['google_1980']['kfold_splits'][
        lead_time][return_period].dropna().rename('kfold_splits')
], axis=1).dropna()

corr_matrix = pd.concat([regression_attributes, combined_data], axis=1).dropna()
c = corr_matrix.corr()[[metrics_utils.GLOFAS_VARIABLE, 'kfold_splits']].drop(
    [metrics_utils.GLOFAS_VARIABLE, 'kfold_splits'])
cabs = c.abs()

im = bigax.imshow(
    c.loc[top_features, [metrics_utils.GLOFAS_VARIABLE, 'kfold_splits']], 
    cmap='PiYG',
    vmin=-0.25, vmax=0.25,
)
bigax.set_xticks(
    [0,1], 
    ['GloFAS', 'AI Model'],
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)
bigax.set_yticks(
    range(len(top_features)),
    top_features,
    fontsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']
)
cbar = plt.colorbar(im, ax=bigax)
cbar.set_label(
    'Correlations between F1 scores and Catchment Attributes', 
    rotation=90, 
    fontsize=evaluation_utils.NATURE_FONT_SIZES['axis_label']
)
cbar.ax.tick_params(labelsize=evaluation_utils.NATURE_FONT_SIZES['tick_labels']) 

plt.tight_layout()

evaluation_utils.save_figure(data_paths.PREDICATABILITY_CONFUSION_MATRICES_FILENAME)